In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import gcf

In [2]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd 
import sys
import pickle
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.metrics import pairwise_distances
from sklearn.kernel_ridge import KernelRidge
import math

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

from mcmodels.core import VoxelModelCache
from mcmodels.core.utils import get_structure_id,get_ordered_summary_structures
from mcmodels.core.connectivity_data import get_connectivity_data
from mcmodels.models.voxel.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists
from mcmodels.utils import nonzero_unique #, unionize
#from mcmodels.models.crossvalidation import get_best_hyperparameters,get_loss_best_hyp,get_loss#get_loocv_predictions,get_loss#get_best_hyperparameters,get_loss_best_hyp,get_loocv_predictions,get_loss
from mcmodels.core.utils import get_leaves_ontologicalorder, get_indices, get_indices2,get_eval_indices,screen_index_matrices,screen_index_matrices2,screen_index_matrices3,get_indices_2ormore#get_cre_status,get_minorstructure_dictionary,get_leaves_ontologicalorder
#from mcmodels.regressors import NadarayaWatson
#from mcmodels.core.plotting import plot_loss_surface,plot_loss_scatter
def fix_pdcsv(csv):
    
    csv_rownames = np.asarray(csv.iloc[:,0])
    csv = csv.iloc[:,1:]
    csv.index = csv_rownames
    return(csv)

from mcmodels.core import Mask
def get_regional_centroids(cache,sid,hemisphere_id):
    
    mask = Mask.from_cache(cache, structure_ids=[sid], hemisphere_id=hemisphere_id)
    centroid_loc = np.mean(mask.coordinates, axis = 0)
    
    return(centroid_loc)

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
#read data
TOP_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
INPUT_JSON = os.path.join(TOP_DIR, 'input_011520.json')
EXPERIMENTS_EXCLUDE_JSON = os.path.join(TOP_DIR, 'experiments_exclude.json')
FILE_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
OUTPUT_DIR = os.path.join(FILE_DIR, 'output')

input_data = ju.read(INPUT_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)

#its unclear why the hyperparameters are loaded from the output directory
cache = VoxelModelCache(manifest_file=manifest_file)
major_structures = input_data.get('structures')
major_structure_ids = [get_structure_id(cache, s) for s in major_structures]
data_info = pd.read_excel('/Users/samsonkoelle/alleninstitute/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)
ontological_order = get_ordered_summary_structures(cache, 167587189)

mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}

#regionalize voxel model: compare with regional model
#regional parameters
cre = None
eid_set=None
high_res=False
threshold_injection = False

COARSE_STRUCTURE_SET_ID = 2
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
tree = cache.get_structure_tree()
default_structures = tree.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
default_structure_ids = [st['id'] for st in default_structures if st['id'] != 934]
#cre= True

annotation/ccf_2017


In [4]:
connectivity_data = get_connectivity_data(cache, major_structure_ids[:12], experiments_exclude, remove_injection = False)

connectivity_data.get_injection_hemisphere_ids()
connectivity_data.align()
connectivity_data.get_centroids()
connectivity_data.get_data_matrices(default_structure_ids)
connectivity_data.get_crelines(data_info)
with open('data/info/leafs.pickle', 'rb') as handle:
    leafs = pickle.load(handle)
    
connectivity_data.ai_map = ai_map
connectivity_data.get_summarystructures(data_info)
connectivity_data.leafs = leafs

512
703
1089
1097
315
313
354
698
771
803
477
549


In [5]:
wt_conn = pd.read_csv('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/connectivities/el_leafsurf_leafsmth_leafleaf_C57BL6J0428.csv',  header=[0,1])
wt_conn = fix_pdcsv(wt_conn)
wt_conn.shape

(564, 1123)

In [30]:
ic_targ_names = np.asarray(list(wt_conn.columns))[:,1]
ic_targ_id = np.asarray([ai_map[ic_targ_names[i]] for i in range(len(ic_targ_names))])

In [18]:
columns = np.asarray(list(wt_conn.columns))
ipsi_inds = np.where(columns[:,0] == 'ipsi')[0]
contra_inds = np.where(columns[:,0] == 'contra')[0]
ipsi_names = columns[ipsi_inds,1]
contra_names = columns[contra_inds,1]
rows = np.asarray(list(wt_conn.index))

In [31]:
regional_centroids_sources = np.asarray([get_regional_centroids(cache,ai_map[rows[i]],hemisphere_id = 2) for i in range(len(rows))])

# ipsi_names = np.asarray(list(wt_conn.columns))[:291][:,1]
# contra_names = np.asarray(list(wt_conn.columns))[291:][:,1]
regional_centroids_ipsitargets = np.asarray([get_regional_centroids(cache,ai_map[ipsi_names[i]],hemisphere_id = 2) for i in range(len(ipsi_inds))])
regional_centroids_contratargets = np.asarray([get_regional_centroids(cache,ai_map[contra_names[i]],hemisphere_id = 1) for i in range(len(contra_inds))])

#nreg = wt_conn_oo.shape[0]
#crow = np.zeros((nreg,4), dtype  =float)
# for i in range(nreg):
#     crow[i,:] = np.asarray(cs_col_ipsi_dict[major_ontoloogy[i]])

In [32]:
regional_centroids_targets = np.vstack([regional_centroids_ipsitargets,regional_centroids_contratargets])

ntarg = wt_conn.shape[1]
nsource = regional_centroids_sources.shape[0]
distances = np.zeros((nsource, ntarg))
for i in range(nsource):
    for j in range(ntarg):
        distances[i,j] = np.linalg.norm(regional_centroids_sources[i] - regional_centroids_targets[j])

In [7]:
from mcmodels.utils import get_aligned_ids

In [31]:
ontological_order_leaves = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/ontological_order_leaves_v3.npy')
ontological_order = np.load('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/info/ontological_order_v3.npy')
#ontological_order_leaves_majors = get_aligned_ids(st,ontological_order_leaves,major_structure_ids)
ontological_order_i_majors = get_aligned_ids(st,ontological_order_leaves,major_structure_ids)
ontological_order_ic_majors = get_aligned_ids(st,ic_targ_id,major_structure_ids)

In [32]:
ontological_order_ic_majors

array([315, 315, 315, ..., 512, 512, 512])

In [19]:
#ontological_order_leaves_names = np.asarray([ia_map[ontological_order_leaves[i]] for i in range(len(ontological_order_leaves))])

In [13]:
# contra_inds = np.where(np.asarray(list(wt_conn.columns))[:,0] == 'contra')[0]
# ipsi_inds = np.where(np.asarray(list(wt_conn.columns))[:,0] == 'ipsi')[0]

# contra_names = ontological_order_leaves_names[np.in1d(ontological_order_leaves_names,np.asarray(list(wt_conn.columns))[:,1][contra_inds])]

# ipsi_names = ontological_order_leaves_names[np.in1d(ontological_order_leaves_names,np.asarray(list(wt_conn.columns))[:,1][ipsi_inds])]

# icomb = np.asarray([np.repeat('ipsi',ipsi_names.shape[0]),ipsi_names])
# icomb = np.asarray(icomb)
# tuples2 = list(zip(*icomb))
# ipsi_multi = pd.MultiIndex.from_tuples(tuples2, names=['first', 'second'])
# #contra_inds = cnam_multi[~np.in1d(cnam_multi,wt_conn.columns)]

# ccomb = np.asarray([np.repeat('contra',contra_names.shape[0]),contra_names])
# ccomb = np.asarray(ccomb)
# tuples2 = list(zip(*ccomb))
# cnam_multi = pd.MultiIndex.from_tuples(tuples2, names=['first', 'second'])
# #contra_multi = cnam_multi[~np.in1d(cnam_multi,wt_conn.columns)]

# comb = np.hstack([icomb,ccomb])
# tuples = list(zip(*comb))
# multi = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])

In [ ]:
# ool_ic = np.asarray([ai_map[comb[1][i]] for i in range(len(comb[1]))])
# ool_ic_majors = get_aligned_ids(st, ool_ic,major_structure_ids)
# ool_ic_major_names = np.asarray([ia_map[ool_ic_majors[i]] for i in range(len(oos_ic_majors))])

In [33]:
# ool_i = np.asarray([ai_map[comb[1][i]] for i in range(len(comb[1]))])
# ool_i_majors = get_aligned_ids(st, ool_ic,major_structure_ids)
# ool_i_major_names = np.asarray([ia_map[ool_ic_majors[i]] for i in range(len(oos_ic_majors))])

In [34]:
#major_ontoloogy_summary = np.load('/Users/samsonkoelle/ontological_order_summary_majors_names.npy')
#ontological_order_leaves_majors_names = np.asarray([ia_map[ontological_order_leaves_majors[i]] for i in range(len(ontological_order_leaves_majors))])
#ontological_order_summary_majors = get_aligned_ids(ontological_order,major_structure_ids)
#ontological_order_summary_majors_names = np.asarray([ia_map[ontological_order_summary_majors[i]] for i in range(len(ontological_order_summary_majors))])


In [35]:
#oos_ic_major_names.shape

In [36]:
#np.save('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/analyses/paper/getinfo/ontological_order_summary_majors_names_v3', ontological_order_summary_majors_names)
np.save('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/analyses/paper/getinfo/ool_ic_majors_names_v3_0610', ontological_order_ic_majors)
np.save('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/analyses/paper/getinfo/ool_i_majors_names_v3_0610', ontological_order_i_majors)

In [ ]:
np.save('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/analyses/paper/getinfo/distances_leaf_leaf', distances)